# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [19]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [21]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Instantiate a text generation pipeline using the tokenizer and model.

In [22]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [23]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Hyvin, kiitos.”\n”Kiva kuulla.”\n”Kuule, minulla on sinulle asiaa.”\n'}]


Just print the text

In [24]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [26]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee? kysyi Heikki yhtäkkiä astuessaan taloon sisään kantaen kahvipöytä-tarvikkeita käsissään sisään tultuaan.
(Ryökäle luuli varmasti hänen tulevan meille istumaan)




## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load model
MODEL_NAME = 'Qwen/Qwen2.5-1.5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device='cpu'
)

def generate_response(prompt, max_tokens=10):
    output = pipe(prompt, max_new_tokens=max_tokens, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    response = output[0]['generated_text'][len(prompt):].strip()
    
    return response

Device set to use cpu


In [17]:
# Task 1: Binary Sentiment Classification
print("=== SENTIMENT CLASSIFICATION ===")

# First test case
zero_shot_sentiment = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: This movie was absolutely amazing and delightful!
Sentiment:"""

print("Zero-shot:")
print(generate_response(zero_shot_sentiment))

one_shot_sentiment = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: I hate this terrible product.
Sentiment: negative

Text: This movie was absolutely amazing and delightful!
Sentiment:"""

print("\nOne-shot:")
print(generate_response(one_shot_sentiment))

two_shot_sentiment = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: I hate this terrible product.
Sentiment: negative

Text: This book is wonderful and inspiring.
Sentiment: positive

Text: This movie was absolutely amazing and delightful!
Sentiment:"""

print("\nTwo-shot:")
print(generate_response(two_shot_sentiment))

=== SENTIMENT CLASSIFICATION ===
Zero-shot:
positive

The sentiment expressed in the given text is

One-shot:
positive

The sentiment expressed in the given text is

One-shot:
positive

Text: The weather is so nice today

Two-shot:
positive

Text: The weather is so nice today

Two-shot:
positive

Text: The food at this restaurant is
positive

Text: The food at this restaurant is


In [13]:
# Second sentiment test case
print("\n--- Second sentiment test ---")

zero_shot_sentiment2 = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: The weather is horrible and ruined my day.
Sentiment:"""

print("Zero-shot:")
print(generate_response(zero_shot_sentiment2))

one_shot_sentiment2 = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: I hate this terrible product.
Sentiment: negative

Text: The weather is horrible and ruined my day.
Sentiment:"""

print("\nOne-shot:")
print(generate_response(one_shot_sentiment2))

two_shot_sentiment2 = """Classify the sentiment of the following text as either 'positive' or 'negative'. Only respond with one word.

Text: I hate this terrible product.
Sentiment: negative

Text: This book is wonderful and inspiring.
Sentiment: positive

Text: The weather is horrible and ruined my day.
Sentiment:"""

print("\nTwo-shot:")
print(generate_response(two_shot_sentiment2))


--- Second sentiment test ---
Zero-shot:
negative.

One-shot:
negative.

One-shot:
negative

Two-shot:
negative

Two-shot:
negative
negative


In [14]:
# Task 2: Person Name Recognition
print("=== PERSON NAME RECOGNITION ===")

# First test case
zero_shot_names = """List the person names occurring in the following texts.

Text: John Smith met with Sarah Johnson at the conference yesterday.
Names:"""

print("Zero-shot:")
print(generate_response(zero_shot_names))

one_shot_names = """List the person names occurring in the following texts.

Text: Michael Brown and Lisa Davis went to the store.
Names: Michael Brown, Lisa Davis

Text: John Smith met with Sarah Johnson at the conference yesterday.
Names:"""

print("\nOne-shot:")
print(generate_response(one_shot_names))

two_shot_names = """List the person names occurring in the following texts.

Text: Michael Brown and Lisa Davis went to the store.
Names: Michael Brown, Lisa Davis

Text: Professor Wilson taught the class while Emma Thompson took notes.
Names: Wilson, Emma Thompson

Text: John Smith met with Sarah Johnson at the conference yesterday.
Names:"""

print("\nTwo-shot:")
print(generate_response(two_shot_names))

=== PERSON NAME RECOGNITION ===
Zero-shot:
['John', 'Smith', 'Sarah', '

One-shot:
['John', 'Smith', 'Sarah', '

One-shot:
John Smith, Sarah Johnson

Two-shot:
John Smith, Sarah Johnson

Two-shot:
John Smith, Sarah Johnson
John Smith, Sarah Johnson


In [15]:
# Second name recognition test case
print("\n--- Second name recognition test ---")

zero_shot_names2 = """List the person names occurring in the following texts.

Text: Dr. Anderson spoke with Maria Garcia about the research project.
Names:"""

print("Zero-shot:")
print(generate_response(zero_shot_names2))

one_shot_names2 = """List the person names occurring in the following texts.

Text: Michael Brown and Lisa Davis went to the store.
Names: Michael Brown, Lisa Davis

Text: Dr. Anderson spoke with Maria Garcia about the research project.
Names:"""

print("\nOne-shot:")
print(generate_response(one_shot_names2))

two_shot_names2 = """List the person names occurring in the following texts.

Text: Michael Brown and Lisa Davis went to the store.
Names: Michael Brown, Lisa Davis

Text: Professor Wilson taught the class while Emma Thompson took notes.
Names: Wilson, Emma Thompson

Text: Dr. Anderson spoke with Maria Garcia about the research project.
Names:"""

print("\nTwo-shot:")
print(generate_response(two_shot_names2))


--- Second name recognition test ---
Zero-shot:
Dr. Anderson, Maria Garcia

One-shot:
Dr. Anderson, Maria Garcia

One-shot:
Dr. Anderson, Maria Garcia

Two-shot:
Dr. Anderson, Maria Garcia

Two-shot:
Anderson, Maria Garcia
Anderson, Maria Garcia


In [18]:
# Task 3: Two-digit Addition
print("=== TWO-DIGIT ADDITION ===")

# First test case
zero_shot_math = """This is a first grade math exam.

Problem: 23 + 45 = 
Answer:"""

print("Zero-shot:")
print(generate_response(zero_shot_math))

one_shot_math = """This is a first grade math exam.

Problem: 12 + 34 = 46

Problem: 23 + 45 = 
Answer:"""

print("\nOne-shot:")
print(generate_response(one_shot_math))

two_shot_math = """This is a first grade math exam.

Problem: 12 + 34 = 46
Problem: 67 + 21 = 88

Problem: 23 + 45 = 
Answer:"""

print("\nTwo-shot:")
print(generate_response(two_shot_math))

=== TWO-DIGIT ADDITION ===
Zero-shot:
68

What are the steps to solve

One-shot:
68

What are the steps to solve

One-shot:
68

Problem: 78 -

Two-shot:
68

Problem: 78 -

Two-shot:
68

What is the next problem in
68

What is the next problem in


In [19]:
# Second math test case
print("\n--- Second math test ---")

zero_shot_math2 = """This is a first grade math exam.

Problem: 56 + 37 = 
Answer:"""

print("Zero-shot:")
print(generate_response(zero_shot_math2))

one_shot_math2 = """This is a first grade math exam.

Problem: 12 + 34 = 46

Problem: 56 + 37 = 
Answer:"""

print("\nOne-shot:")
print(generate_response(one_shot_math2))

two_shot_math2 = """This is a first grade math exam.

Problem: 12 + 34 = 46
Problem: 67 + 21 = 88

Problem: 56 + 37 = 
Answer:"""

print("\nTwo-shot:")
print(generate_response(two_shot_math2))


--- Second math test ---
Zero-shot:
93

What are the steps to solve

One-shot:
93

What are the steps to solve

One-shot:
93

Problem: 89 +

Two-shot:
93

Problem: 89 +

Two-shot:
93

What is the next problem in
93

What is the next problem in


**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**